In [ ]:
import pandas as pd
import time
import requests
import pickle
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from google.colab import output
import datetime
from datetime import datetime as dt




In [ ]:
def get_price(stock):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    url = f"https://finance.yahoo.com/quote/{stock}?p={stock}&.tsrc=fin-srch"
    r = requests.get(url, headers=headers)
    soup = bs(r.content, "html.parser")

    try:
        price = soup.find("fin-streamer", attrs={"data-field": "regularMarketPrice", "class":"Fw(b) Fz(36px) Mb(-4px) D(ib)"}).get_text()
        price = float(price.replace(",", ""))
        return price

    except Exception as e:
        print(stock)
        print(e)
        return 0


In [ ]:
portfolio = {
    "SPY": 0.2614,
    "BABA": 0.0334,
    "FB": 0.0773,
    "TSLA": 0.1066,
    "NFLX": 0.0217,
    "JPM": 0.0158,
    "INDS": 0.0824,
    "AAPL": 0.0109,
    "TCEHY": 0.0186,
    "PFE": 0.0691,
    "PBW": 0.0315,
    "PTF": 0.0347,
    "BAC": 0.0166,
    "AMZN": 0.0327,
    "MSFT": 0.0164,
    "CVS": 0.0327,
    "BTC": 0.0555,
    "DOGE-USD": 0.0794,
    "ETC-USD": 0.0150,
    "ETH-USD": 0.0486
}

def get_dataframe(stock, anaysis_period = 3650):
    # A huge epoch that allows most up to date info
    current_epoch = 3000000000
    past_epoch = int((dt.now() - datetime.timedelta(anaysis_period)).timestamp())
    url = f"https://query1.finance.yahoo.com/v7/finance/download/{stock}?period1={past_epoch}&period2={current_epoch}&interval=1d&events=history&includeAdjustedClose=true"
    df = pd.read_csv(url)
    return df

previous_standing = {'net_worth': 0}
for p in portfolio:
    previous_standing[p] = get_dataframe(p).Close.to_list()[-2]
    previous_standing['net_worth'] += previous_standing[p] * portfolio[p]
    time.sleep(1)

HTTPError: ignored

In [ ]:
networth_composition = {"net_worth": []}
for p in portfolio:
    networth_composition[p] = []


def update_networth():
    net_worth = 0
    for c in networth_composition:
        if c != "net_worth":
            price = get_price(c)
            while price == 0:
                price = get_price(c)
            change = (price - previous_standing[c]) / previous_standing[c]
            networth_composition[c].append(change)
            net_worth += price * portfolio[c]
            print("stock\tprice\tchange")
            print(f"{c}\t{price}\t{change}")

    networth_composition["net_worth"].append((net_worth - previous_standing["net_worth"]) / previous_standing["net_worth"])

def animate(i):
    plt.rcParams['figure.figsize'] = [20, 10]
    plt.clf()
    update_networth()
    for c in networth_composition:
        if c == "net_worth":
            plt.plot(range(i), networth_composition[c], "o",label=c)
        else:
            plt.plot(range(i), networth_composition[c], label=c)

    plt.plot(range(i), [0 for _ in range(i)], "o")
    plt.legend()
    output.clear()
    plt.show()


i = 1
while True:
    animate(i)
    i += 1
